In [2]:
pip install opencv-python numpy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
import os

prototxt_path = r'C:/Users/Lakshita/Desktop/machine_leaning/Computer Vision/Cap_live _detection/deploy.prototxt'
caffemodel_path = r'C:/Users/Lakshita/Desktop/machine_leaning/Computer Vision/Cap_live _detection/res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

cap_images = []
cap_folder = r'C:/Users/Lakshita/Desktop/machine_leaning/Computer Vision/Cap_live _detection/images/'
for filename in os.listdir(cap_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img = cv2.imread(os.path.join(cap_folder, filename))
        img = cv2.resize(img, (100, 100))
        cap_images.append(img)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (startX, startY, endX, endY) = box.astype("int")

            head_height = endY - startY
            cap_region_height = int(head_height * 0.1)
            startY = max(0, startY - int(head_height * 0.1))

            cap_region = frame[startY:endY, startX:endX]

            cap_detected = False

            threshold = 0.8
            for cap_image in cap_images:
                if cap_region.shape[0] < cap_image.shape[0] or cap_region.shape[1] < cap_image.shape[1]:
                    continue

                result = cv2.matchTemplate(cap_region, cap_image, cv2.TM_CCOEFF_NORMED)
                loc = np.where(result >= threshold)

                if loc[0].size > 0:
                    cap_detected = True
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 0, 0), 2)
                    cv2.putText(frame, "Cap Detected", (startX, startY - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
                    break

    cv2.imshow('Live Feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import os
import tkinter as tk
from tkinter import Label, Button
from PIL import Image, ImageTk

# Load the pre-trained MobileNet SSD model for person detection
prototxt_path = r'C:/Users/Lakshita/Desktop/machine_leaning/Computer Vision/Cap_live _detection/deploy.prototxt'
caffemodel_path = r'C:/Users/Lakshita/Desktop/machine_leaning/Computer Vision/Cap_live _detection/res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Load cap images from dataset
cap_images = []
cap_folder = r'C:/Users/Lakshita/Desktop/machine_leaning/Computer Vision/Cap_live _detection/images/'
for filename in os.listdir(cap_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img = cv2.imread(os.path.join(cap_folder, filename))
        img = cv2.resize(img, (100, 100))
        cap_images.append(img)

# Function for starting video capture
def video_stream():
    global cap_detected

    ret, frame = cap.read()
    if not ret:
        return

    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (startX, startY, endX, endY) = box.astype("int")

            head_height = endY - startY
            cap_region_height = int(head_height * 0.2)
            startY = max(0, startY - int(head_height * 0.2))

            cap_region = frame[startY:endY, startX:endX]

            cap_detected = False

            threshold = 0.8
            for cap_image in cap_images:
                if cap_region.shape[0] < cap_image.shape[0] or cap_region.shape[1] < cap_image.shape[1]:
                    continue

                result = cv2.matchTemplate(cap_region, cap_image, cv2.TM_CCOEFF_NORMED)
                loc = np.where(result >= threshold)

                if loc[0].size > 0:
                    cap_detected = True
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 0, 0), 2)
                    cv2.putText(frame, "Cap Detected", (startX, startY - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
                    break

    # Convert the frame to display in Tkinter
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame)
    imgtk = ImageTk.PhotoImage(image=img)
    label.imgtk = imgtk
    label.configure(image=imgtk)
    label.after(10, video_stream)

# Initialize the webcam feed
cap = cv2.VideoCapture(0)

# Create the main window
window = tk.Tk()
window.title("Cap Detection Application")

# Create a Label to hold the video stream
label = Label(window)
label.pack()

# Start the video stream
video_stream()

# Add a button to quit the application
quit_button = Button(window, text="Quit", command=window.quit)
quit_button.pack()

# Start the Tkinter event loop
window.mainloop()

# Release resources
cap.release()
cv2.destroyAllWindows()
